<a href="https://colab.research.google.com/github/theDikshaSaxena/Recontextualization-Detection/blob/main/FactComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
! pip install torchtext==0.10.1

import torch
device = torch.device("cuda")
torch.cuda.init()
import numpy as np
import time
import math

In [1]:
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.


In [2]:
!python -m spacy download en_core_web_sm

2023-07-31 09:06:00.614781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 09:06:01.802741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.5.0
    Uninstalling en-core-web-sm-3.5.0:
      Successfully uninstalled en-core-web-sm-3.5.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
import spacy

# Load the English language model for spaCy
nlp = spacy.load("en_core_web_sm")

# Function to extract entities of specific types from a paragraph
def extract_specific_entities(paragraph, entity_types):
    doc = nlp(paragraph)
    entities = []
    for ent in doc.ents:
        if ent.label_ in entity_types:
            entities.append((ent.text, ent.label_))
    return entities

# Example paragraphs
paragraph1 = "Rescue workers are surveying the area after a suspected dirty bomb explosion. I can't believe Russia would commit these war crimes so openly. Its time for the world to stop these attacks now before its to late."
paragraph2 = "Thursday, killing at least five people, hours ahead of the first face-to-face meeting since the start of the war between the Turkish and Ukrainian leaders.Moscow meanwhile denied it had deployed any heavy weapons at the Russian-controlled Zaporizhzhia nuclear power plant in southern Ukraine where a recent escalation in fighting has increased fears of a nuclear disaster.The head of the Kharkiv region Oleg Synegubov said Moscows forces had launched eight missiles from Russian territory at around 0430 local time (0130 GMT) striking across the city.Three people died, including a child. Eight people, including two children, were rescued, the emergency services said."

# Extract entities of specific types from the paragraphs
entity_types_to_compare = ['GPE', 'PERSON', 'EVENT']  # GPE: Geopolitical Entity, PERSON: Person, EVENT: Event
entities1 = extract_specific_entities(paragraph1, entity_types_to_compare)
entities2 = extract_specific_entities(paragraph2, entity_types_to_compare)

# Print the entities for each paragraph
print("Entities in Paragraph 1:")
for entity, label in entities1:
    print(f"{entity} - {label}")

print("\nEntities in Paragraph 2:")
for entity, label in entities2:
    print(f"{entity} - {label}")

# Create sets of entities for comparison
entities_set1 = set(entities1)
entities_set2 = set(entities2)

# Find common entities in both paragraphs
common_entities = entities_set1.intersection(entities_set2)
print("\nCommon Entities:")
for entity, label in common_entities:
    print(f"{entity} - {label}")

# Find unique entities in each paragraph
unique_entities_paragraph1 = entities_set1 - common_entities
unique_entities_paragraph2 = entities_set2 - common_entities

print("\nUnique Entities in Paragraph 1:")
for entity, label in unique_entities_paragraph1:
    print(f"{entity} - {label}")

print("\nUnique Entities in Paragraph 2:")
for entity, label in unique_entities_paragraph2:
    print(f"{entity} - {label}")

# Check if facts for entities are consistent across both paragraphs
consistent_facts = []
inconsistent_facts = []
for entity1 in entities_set1:
    for entity2 in entities_set2:
        if entity1[0] == entity2[0]:  # Check if entity text matches
            consistent_facts.append(entity1)
            break
    else:
        inconsistent_facts.append(entity1)

print("\nConsistent Facts:")
for entity, label in consistent_facts:
    print(f"{entity} - {label}")

print("\nInconsistent Facts:")
for entity, label in inconsistent_facts:
    print(f"{entity} - {label}")


Entities in Paragraph 1:
Russia - GPE

Entities in Paragraph 2:
Moscow - GPE
Zaporizhzhia - PERSON
Ukraine - GPE
Oleg Synegubov - PERSON

Common Entities:

Unique Entities in Paragraph 1:
Russia - GPE

Unique Entities in Paragraph 2:
Ukraine - GPE
Oleg Synegubov - PERSON
Moscow - GPE
Zaporizhzhia - PERSON

Consistent Facts:

Inconsistent Facts:
Russia - GPE


In [7]:
import spacy
import numpy as np
from scipy.spatial.distance import cosine

# Load the English language model for spaCy
nlp = spacy.load("en_core_web_sm")

# Function to calculate sentence embeddings using word embeddings
def sentence_embedding(sentence):
    doc = nlp(sentence)
    # Use average word vector as sentence embedding
    return np.mean([token.vector for token in doc if not token.is_stop], axis=0)

# Example paragraphs
paragraph1 = "Rescue workers are surveying the area after a suspected dirty bomb explosion. I can't believe Russia would commit these war crimes so openly. Its time for the world to stop these attacks now before its to late."
paragraph2 = "Thursday, killing at least five people, hours ahead of the first face-to-face meeting since the start of the war between the Turkish and Ukrainian leaders.Moscow meanwhile denied it had deployed any heavy weapons at the Russian-controlled Zaporizhzhia nuclear power plant in southern Ukraine where a recent escalation in fighting has increased fears of a nuclear disaster.The head of the Kharkiv region Oleg Synegubov said Moscows forces had launched eight missiles from Russian territory at around 0430 local time (0130 GMT) striking across the city.Three people died, including a child. Eight people, including two children, were rescued, the emergency services said."

# Calculate sentence embeddings
embedding1 = sentence_embedding(paragraph1)
embedding2 = sentence_embedding(paragraph2)

# Compute cosine similarity between the embeddings
similarity = 1 - cosine(embedding1, embedding2)

# Threshold for considering sentences consistent
consistency_threshold = 0.8

if similarity >= consistency_threshold:
    print("The facts in the paragraphs are consistent.")
else:
    print("The facts in the paragraphs are not consistent.")


The facts in the paragraphs are not consistent.


In [10]:
import spacy
import numpy as np
from scipy.spatial.distance import cosine

# Load the English language model for spaCy
nlp = spacy.load("en_core_web_sm")

# Function to calculate sentence embeddings using word embeddings
def sentence_embedding(sentence):
    doc = nlp(sentence)
    # Use average word vector as sentence embedding
    return np.mean([token.vector for token in doc if not token.is_stop], axis=0)

# Example paragraphs
paragraph1 = "So senseless to target a building like a hostel. Multiple people killed. Hopefully the world will take notice of these tragic events soon."
paragraph2 = "Thursday, killing at least five people, hours ahead of the first face-to-face meeting since the start of the war between the Turkish and Ukrainian leaders.Moscow meanwhile denied it had deployed any heavy weapons at the Russian-controlled Zaporizhzhia nuclear power plant in southern Ukraine where a recent escalation in fighting has increased fears of a nuclear disaster.The head of the Kharkiv region Oleg Synegubov said Moscows forces had launched eight missiles from Russian territory at around 0430 local time (0130 GMT) striking across the city.Three people died, including a child. Eight people, including two children, were rescued, the emergency services said."

# Calculate sentence embeddings
embedding1 = sentence_embedding(paragraph1)
embedding2 = sentence_embedding(paragraph2)

# Compute cosine similarity between the embeddings
similarity = 1 - cosine(embedding1, embedding2)

# Threshold for considering sentences consistent
consistency_threshold = 0.8

if similarity >= consistency_threshold:
    print("The facts in the paragraphs are consistent.")
else:
    print("The facts in the paragraphs are not consistent.")


The facts in the paragraphs are not consistent.


In [11]:
import spacy
import numpy as np
from scipy.spatial.distance import cosine

# Load the English language model for spaCy
nlp = spacy.load("en_core_web_sm")

# Function to calculate sentence embeddings using word embeddings
def sentence_embedding(sentence):
    doc = nlp(sentence)
    # Use average word vector as sentence embedding
    return np.mean([token.vector for token in doc if not token.is_stop], axis=0)

# Function to compare the facts in two paragraphs
def compare_facts(paragraph1, paragraph2, consistency_threshold=0.8):
    # Calculate sentence embeddings
    embedding1 = sentence_embedding(paragraph1)
    embedding2 = sentence_embedding(paragraph2)

    # Compute cosine similarity between the embeddings
    similarity = 1 - cosine(embedding1, embedding2)

    # Check if the facts are consistent
    if similarity >= consistency_threshold:
        return "The facts in the paragraphs are consistent."
    else:
        return "The facts in the paragraphs are not consistent."

# Example paragraphs
paragraph1 = "So senseless to target a building like a hostel. Multiple people killed. Hopefully the world will take notice of these tragic events soon."
paragraph2 = "Thursday, killing at least five people, hours ahead of the first face-to-face meeting since the start of the war between the Turkish and Ukrainian leaders.Moscow meanwhile denied it had deployed any heavy weapons at the Russian-controlled Zaporizhzhia nuclear power plant in southern Ukraine where a recent escalation in fighting has increased fears of a nuclear disaster.The head of the Kharkiv region Oleg Synegubov said Moscows forces had launched eight missiles from Russian territory at around 0430 local time (0130 GMT) striking across the city.Three people died, including a child. Eight people, including two children, were rescued, the emergency services said."

# Compare the facts in the paragraphs
result = compare_facts(paragraph1, paragraph2)

# Print the result
print(result)


The facts in the paragraphs are not consistent.
